In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('../data/Bicycle_Thefts.csv')
print (df.keys())
print (type(df.keys()))

Index(['OBJECTID', 'EVENT_UNIQUE_ID', 'PRIMARY_OFFENCE', 'OCC_DATE',
       'OCC_YEAR', 'OCC_MONTH', 'OCC_DOW', 'OCC_DAY', 'OCC_DOY', 'OCC_HOUR',
       'REPORT_DATE', 'REPORT_YEAR', 'REPORT_MONTH', 'REPORT_DOW',
       'REPORT_DAY', 'REPORT_DOY', 'REPORT_HOUR', 'DIVISION', 'LOCATION_TYPE',
       'PREMISES_TYPE', 'BIKE_MAKE', 'BIKE_MODEL', 'BIKE_TYPE', 'BIKE_SPEED',
       'BIKE_COLOUR', 'BIKE_COST', 'STATUS', 'HOOD_158', 'NEIGHBOURHOOD_158',
       'HOOD_140', 'NEIGHBOURHOOD_140', 'LONG_WGS84', 'LAT_WGS84', 'x', 'y'],
      dtype='object')
<class 'pandas.core.indexes.base.Index'>


In [6]:
import pandas as pd
import numpy as np
from datetime import datetime

def clean_bicycle_data(df):
    """
    Hàm làm sạch dữ liệu về các vụ trộm xe đạp
    """
    # Tạo bản sao của DataFrame để tránh thay đổi dữ liệu gốc
    df_clean = df.copy()
    
    # 1. Chuyển đổi các cột thời gian sang định dạng datetime
    datetime_columns = ['OCC_DATE', 'REPORT_DATE']
    for col in datetime_columns:
        df_clean[col] = pd.to_datetime(df_clean[col])
    
    # 2. Chuẩn hóa các giá trị trong cột BIKE_COLOUR
    df_clean['BIKE_COLOUR'] = df_clean['BIKE_COLOUR'].str.strip().str.upper()
    color_mapping = {
        'BLK': 'BLACK',
        'WHI': 'WHITE',
        'SILRED': 'SILVER RED',
        '': 'UNKNOWN'
    }
    df_clean['BIKE_COLOUR'] = df_clean['BIKE_COLOUR'].map(lambda x: color_mapping.get(x, x))
    
    # 3. Xử lý giá trị thiếu trong BIKE_COST
    df_clean['BIKE_COST'] = pd.to_numeric(df_clean['BIKE_COST'], errors='coerce')
    # Điền giá trị thiếu bằng median của loại xe tương ứng
    df_clean['BIKE_COST'] = df_clean.groupby('BIKE_TYPE')['BIKE_COST'].transform(
        lambda x: x.fillna(x.median())
    )
    # Nếu vẫn còn giá trị NA (do cả nhóm đều NA), điền bằng median tổng thể
    df_clean['BIKE_COST'] = df_clean['BIKE_COST'].fillna(df_clean['BIKE_COST'].median())
    
    # 4. Chuẩn hóa BIKE_SPEED thành numeric
    df_clean['BIKE_SPEED'] = pd.to_numeric(df_clean['BIKE_SPEED'], errors='coerce')
    df_clean['BIKE_SPEED'] = df_clean['BIKE_SPEED'].fillna(df_clean['BIKE_SPEED'].median())
    
    # 5. Chuẩn hóa BIKE_TYPE
    df_clean['BIKE_TYPE'] = df_clean['BIKE_TYPE'].fillna('UNKNOWN')
    bike_type_mapping = {
        'MT': 'MOUNTAIN',
        'RG': 'ROAD',
        'RC': 'RACING',
        'OT': 'OTHER'
    }
    df_clean['BIKE_TYPE'] = df_clean['BIKE_TYPE'].map(lambda x: bike_type_mapping.get(x, x))
    
    # 6. Làm sạch và chuẩn hóa các cột địa điểm
    df_clean['LOCATION_TYPE'] = df_clean['LOCATION_TYPE'].fillna('UNKNOWN')
    df_clean['PREMISES_TYPE'] = df_clean['PREMISES_TYPE'].fillna('UNKNOWN')
    
    # 7. Xử lý tọa độ địa lý
    # Loại bỏ các tọa độ không hợp lệ (nếu có)
    df_clean = df_clean[
        (df_clean['LAT_WGS84'].between(43.5, 44.0)) & 
        (df_clean['LONG_WGS84'].between(-80.0, -79.0))
    ]
    
    # 8. Tạo thêm các trường hữu ích
    df_clean['REPORT_DELAY'] = (df_clean['REPORT_DATE'] - df_clean['OCC_DATE']).dt.total_seconds() / 3600  # Số giờ
    
    # 9. Chuẩn hóa các cột BIKE_MAKE và BIKE_MODEL
    df_clean['BIKE_MAKE'] = df_clean['BIKE_MAKE'].str.strip().str.upper()
    df_clean['BIKE_MODEL'] = df_clean['BIKE_MODEL'].str.strip().str.upper()
    
    # 10. Đảm bảo STATUS chỉ có các giá trị hợp lệ
    valid_statuses = ['STOLEN', 'RECOVERED']
    df_clean['STATUS'] = df_clean['STATUS'].apply(lambda x: x if x in valid_statuses else 'UNKNOWN')
    
    return df_clean

# Làm sạch dữ liệu
df_cleaned = clean_bicycle_data(df)

# Kiểm tra kết quả
print("Thông tin về DataFrame sau khi làm sạch:")
print(df_cleaned.info())

# Kiểm tra các giá trị null còn lại
null_counts = df_cleaned.isnull().sum()
print("\nSố lượng giá trị null trong mỗi cột:")
print(null_counts[null_counts > 0])

df_cleaned.to_csv('data/clean_data.csv', index=False)

Thông tin về DataFrame sau khi làm sạch:
<class 'pandas.core.frame.DataFrame'>
Index: 35795 entries, 0 to 36124
Data columns (total 36 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   OBJECTID           35795 non-null  int64         
 1   EVENT_UNIQUE_ID    35795 non-null  object        
 2   PRIMARY_OFFENCE    35795 non-null  object        
 3   OCC_DATE           35795 non-null  datetime64[ns]
 4   OCC_YEAR           35795 non-null  int64         
 5   OCC_MONTH          35795 non-null  object        
 6   OCC_DOW            35795 non-null  object        
 7   OCC_DAY            35795 non-null  int64         
 8   OCC_DOY            35795 non-null  int64         
 9   OCC_HOUR           35795 non-null  int64         
 10  REPORT_DATE        35795 non-null  datetime64[ns]
 11  REPORT_YEAR        35795 non-null  int64         
 12  REPORT_MONTH       35795 non-null  object        
 13  REPORT_DOW         35795 

In [9]:
df = pd.read_csv('../data/clean_data.csv')
df['STATUS'].value_counts()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_1220\3987911376.py:1: DtypeWarning: Columns (27,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/clean_data.csv')


STATUS
STOLEN       34834
UNKNOWN        587
RECOVERED      374
Name: count, dtype: int64

In [ ]:
for id, row in enumerate(df['PREMISES_TYPE']):
    if id == 'Apartment' or id == 'House':
        df['PREMISES_TYPE'][row] = 'Residential'
    if id == 'Transit':
        df['PREMISES_TYPE'][row] = 'Other'
    if id == "Educational":
        df['PREMISES_TYPE'][row] = 'Commercial'

df.to_csv('../data/clean_data.csv', index=False)

df['PREMISES_TYPE'].value_counts()

PREMISES_TYPE
Residential    14069
Outside        10704
Commercial      5811
Other           5211
Name: count, dtype: int64